In [457]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from ne_tagger import get_tags
import nltk
from nltk.tag import StanfordNERTagger
from keras.preprocessing import sequence
from collections import OrderedDict
from keras.layers import Dense,LSTM,Embedding,Activation,Dropout,Bidirectional
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.optimizers import RMSprop

In [458]:
df = pd.read_csv('news_data_guardian.csv')
df = shuffle(df)
df_a = pd.DataFrame()
store_d = []
def topic_selector(topic):
    for t in topic:
        df_selected=df[df["Headlines"].str.contains(t)==True]
        store_d.append(df_selected["Headlines"].values)
    return store_d
d=topic_selector(['Chelsea','Trump'])

In [459]:
a = []
i=0
for t in d:
    if(i==0):
        a=list(t)
    else:
        a+=list(t)
    i+=1    
a        

['Guus Hiddink warns Antonio Conte of tough job without Chelsea’s old guard',
 'All-seeing Cech lays to rest Arsenal demons of Chelsea’s Mourinho era | Dominic Fifield',
 'John Terry confirmed Chelsea departure due to managerial uncertainty',
 'Football transfer rumours: James Rodríguez to Manchester City or Chelsea?',
 'Vardy and Mahrez score to send Chelsea packing and Leicester top again',
 'Football transfer rumours: Chelsea to move for Martín Cáceres?',
 'Hiddink gets taste of his Chelsea task after Oscar blows late penalty against Watford',
 'Chelsea and Eden Hazard overwhelm Pellegrini’s weakened Manchester City',
 'Manchester City furious over scheduling of FA Cup game against Chelsea',
 'Chelsea’s Guus Hiddink envious of squad options available to PSG',
 'Chelsea 3-0 Burnley: Premier League – as it happened',
 'Antonio Conte: the volcanic manager who will never settle for second best at Chelsea | Paolo Bandini',
 'Chelsea’s José Mourinho fast becoming much ado about nothing | 

In [460]:
test_chelsea = a[:5]
test_trump = a[-15:]
train_headlines = a[5:453]
train_headlines[-1]

'Tottenham prepare for Chelsea having learned from Wembley defeat'

In [461]:
X_t = []
def create_train(headlines):
    for headline in headlines:
        tokens = nltk.word_tokenize(headline)
        tags = nltk.pos_tag(tokens)
        word_input=[]
        for pair in tags:
            if(pair[1]=='NN' or pair[1]=='NNP'):
                #if(pair[0]!=topic):
                word_input.append(pair[0].lower())
        X_t.append(word_input)
    return X_t    

In [462]:
X_train = create_train(a)

In [463]:
len(X_train)

584

In [464]:
text =""
for i in X_train:
    text += " ".join(i)
    text +=" "
text=text.lower()    

In [465]:
text = text.split(' ')

distinct_words = list(set(text))

int_to_word = {}
word_to_int = {}

i=0
for w in distinct_words:
    word_to_int[w]=i
    int_to_word[i]=w
    i+=1

In [466]:
word_to_int
#int_to_word


{'': 0,
 'challenge': 756,
 'touch': 1,
 'phillips': 2,
 'pato': 3,
 'kurt': 4,
 'palmer': 5,
 'packing': 6,
 'costa': 7,
 'allegation': 184,
 'israel': 12,
 'hidalgo': 11,
 'fair': 13,
 'chelsea’': 14,
 'hotspur': 737,
 'malala': 15,
 'edinson': 16,
 'gop': 17,
 'talent': 18,
 'uncertainty': 19,
 'allocation': 20,
 'head': 21,
 'welcome': 24,
 'rob': 23,
 'chelsea': 187,
 'ruben': 26,
 'toronto': 27,
 'party': 786,
 'götze': 28,
 'quality': 29,
 'moussa': 30,
 'luiz': 365,
 'klopp': 31,
 'hernández': 32,
 'response': 35,
 'thing': 34,
 'jerk’': 375,
 'match': 36,
 'horse': 37,
 'outcry': 38,
 'nieto': 40,
 'silence': 45,
 'romania': 42,
 'man': 43,
 '|': 44,
 'review': 46,
 'malaise': 47,
 'clash': 48,
 'reaction': 911,
 'steward’s': 49,
 'laird': 50,
 'gabriella': 52,
 'chairman’s': 53,
 'defeat': 54,
 'hole': 55,
 'conduct': 56,
 'departure': 57,
 'boufal': 1047,
 'ben': 910,
 'road': 59,
 'terror': 64,
 'agüero': 61,
 'nemanja': 62,
 'brom’s': 63,
 'week’': 66,
 'salvini': 8,
 'man

In [467]:
X_train

[['guus', 'hiddink', 'antonio', 'conte', 'job', 'chelsea’s', 'guard'],
 ['cech',
  'arsenal',
  'chelsea’s',
  'mourinho',
  'era',
  '|',
  'dominic',
  'fifield'],
 ['john', 'terry', 'chelsea', 'departure', 'uncertainty'],
 ['football',
  'transfer',
  'james',
  'rodríguez',
  'manchester',
  'city',
  'chelsea'],
 ['vardy', 'mahrez', 'score', 'chelsea', 'packing', 'leicester', 'top'],
 ['football', 'transfer', 'rumours', 'chelsea', 'martín', 'cáceres'],
 ['hiddink', 'taste', 'chelsea', 'task', 'oscar', 'penalty', 'watford'],
 ['chelsea',
  'eden',
  'hazard',
  'overwhelm',
  'pellegrini’s',
  'manchester',
  'city'],
 ['manchester', 'city', 'scheduling', 'fa', 'cup', 'game', 'chelsea'],
 ['chelsea’s', 'guus', 'hiddink', 'squad'],
 ['chelsea', 'burnley', 'premier', 'league', '–'],
 ['antonio', 'conte', 'manager', 'chelsea', '|', 'paolo', 'bandini'],
 ['chelsea’s', 'josé', 'mourinho', 'ado', 'nothing', 'marina', 'hyde'],
 ['chelsea', 'ladies', 'home', 'wolfsburg', 'champions', 'leag

In [468]:
X = np.zeros((len(X_train),20,len(distinct_words)),dtype=np.int32)
y = np.zeros((len(X_train),2),dtype=np.int32)

train = X_train

y[:453,0]=1
y[453:,1]=1

#print(X_train)



In [469]:
for i,sentence in enumerate(train):
    for t,w in enumerate(sentence):
        X[i,t,word_to_int[w]]=1

In [470]:
X_train = sequence.pad_sequences(X,maxlen=8)

In [471]:
X_train.shape

(584, 8, 1112)

In [498]:
y[:-1]

array([[1, 0],
       [1, 0],
       [1, 0],
       ..., 
       [0, 1],
       [0, 1],
       [0, 1]], dtype=int32)

In [499]:
X_te = np.concatenate([X_train[:20],X_train[-10:]])
y_te = np.concatenate([y[:20],y[-10:]])
X_tr = X_train[20:-10]
y_tr = y[20:-10]

print(len(y_tr))
print(len(y_te))

554
30


In [500]:
model = Sequential()
model.add(LSTM(128,input_shape=(8,len(distinct_words)), dropout_W=0.2, dropout_U=0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [501]:
model.fit(X_tr,y_tr,nb_epoch=100,batch_size=32,shuffle=True,validation_data=(X_te, y_te))

Train on 554 samples, validate on 30 samples
Epoch 1/100
554/554 [==============================] - 4s - loss: 0.5680 - acc: 0.7816 - val_loss: 0.6527 - val_acc: 0.6667
Epoch 2/100
554/554 [==============================] - 1s - loss: 0.5268 - acc: 0.7816 - val_loss: 0.6830 - val_acc: 0.6667
Epoch 3/100
554/554 [==============================] - 1s - loss: 0.5318 - acc: 0.7816 - val_loss: 0.7745 - val_acc: 0.6667
Epoch 4/100
554/554 [==============================] - 1s - loss: 0.5338 - acc: 0.7816 - val_loss: 0.6393 - val_acc: 0.6667
Epoch 5/100
554/554 [==============================] - 1s - loss: 0.5340 - acc: 0.7816 - val_loss: 0.7139 - val_acc: 0.6667
Epoch 6/100
554/554 [==============================] - 1s - loss: 0.5335 - acc: 0.7816 - val_loss: 0.6959 - val_acc: 0.6667
Epoch 7/100
554/554 [==============================] - 1s - loss: 0.5245 - acc: 0.7816 - val_loss: 0.8064 - val_acc: 0.6667
Epoch 8/100
554/554 [==============================] - 1s - loss: 0.5387 - acc: 0.7816 

KeyboardInterrupt: 

In [502]:
# s = a[100]
# print(s)
# s=s.lower()

# tokens = nltk.word_tokenize(s)
# tags = nltk.pos_tag(tokens)
# word_input=[]
# for pair in tags:
#     if(pair[1]=='NN' or pair[1]=='NNP'):
#         #if(pair[0]!=topic):
#         word_input.append(pair[0].lower())
# print(word_input)


# #print(s)
# Xp = np.zeros((1,20,len(distinct_words)),dtype=np.int32)
for t,w in enumerate(word_input):
    Xp[0,t,word_to_int[w]]=1
pred_X = sequence.pad_sequences(Xp,maxlen=8)    
    
p = model.predict(X_te)

In [503]:
p

array([[ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740334,  0.14259671],
       [ 0.85740328,

In [497]:
a=[1,2,3,4,5,6,7,8,9,10]
a[-2:]+a[:2]
a[2:-2]

[3, 4, 5, 6, 7, 8]